In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from engine import evaluate
from model import get_model_instance_segmentation
from glob import glob
from dataset import NucleiExpDataset
import torch
import os.path as op
import numpy as np
from skimage import io
import cv2
import utils
EXPERIMENT_PATH ="E:\\Adrien\\data\\Experiment\\"
ANNOTATION_DIR="E:\\Adrien\\data\\Annotation\\"

In [ ]:
def DAPI_image(images,how = 'maximun projection'):
    if how == 'maximum projection':
        return np.max(images, axis=0)
    elif how == 'focus':
        return images[int(images.shape[0]/2)]
    
def load_img(fname, color='RGB'):
    img = cv2.imread(fname)
    img = cv2.cvtColor(img, cv2.COLOR_BGRA2RGB)
    if color == 'GRAY':
        img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB).astype(np.float32)

    return img  

def get_image(path,channel,z_stack):
    z_stack = ('' if z_stack>99 else '0' if z_stack>9 else '00') + str(z_stack)
    return load_img(glob(path+'*'+channel+'*'+str(z_stack)+'*.tif')[0], color='RGB')

def get_images(path,channels,z_stacks):
    if isinstance(channels, str):
        return np.array([get_image(path,channels,z_stack) for z_stack in z_stacks])
    if isinstance(z_stacks, int):
        return {channel : get_image(path,channel,z_stacks) for channel in channels}
    return {channel : np.array([get_image(path,channel,z_stack) for z_stack in z_stacks]) for channel in channels}

In [ ]:

def collate_fn(batch):
    return list(batch)

In [ ]:
import os
datasets_path = glob(EXPERIMENT_PATH+'*')
data_path = datasets_path[2]
position = next(os.walk(data_path))[1]
path_position = {(int(dir_pos.split('_')[-2]),int(dir_pos.split('_')[-1])): dir_pos for dir_pos in position}
data_path = {key: op.join(data_path,value,'') for key, value in path_position.items()}

In [ ]:
op.join(datasets_path[2],'*')

In [ ]:
dataset = NucleiExpDataset()
dataset.load_shapes(data_path.values(),range(29,40))
dataset.prepare()
data_loader = torch.utils.data.DataLoader(
        dataset, batch_size=2, shuffle=False, num_workers=0,
        collate_fn=collate_fn)

In [ ]:
dataset.__getitem__(0).size()

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# our dataset has two classes only - background and person
num_classes = 2

# get the model using our helper function
model = get_model_instance_segmentation(num_classes)

# move model to the right device
model.to(device)

model.load_state_dict(torch.load('model/1.pt'))

In [ ]:
model.eval()
predictions = []
for i, images in enumerate(data_loader):
    print(i)
    c=images
    with torch.no_grad():
        prediction = model([image.to(device) for image in images])
    predictions.extend(prediction)
   

In [ ]:
i=0
image = dataset.__getitem__(i).numpy().transpose((1,2,0))
boxes = predictions[i]['boxes'].cpu().numpy(),
masks = predictions[i]['masks'].cpu().numpy().squeeze(1).transpose((1,2,0))

In [ ]:
from skimage.morphology import closing,square

In [ ]:
shape = (256,256)
labels = []
image, _,_,_ = utils.resize_image(image,min_dim=shape[0],max_dim=shape[1],padding=False)
image = (image*1000).astype(np.uint8)
for i in range(masks.shape[-1]):
    mask = closing(masks[:,:,i].copy(),square(10))
    mask,_,_,_ = utils.resize_image(masks[:,:,i],min_dim=shape[0],max_dim=shape[1],padding=False)
    labels.append((mask>0.5)*(i+1))
label = np.array(labels).max(0).astype(np.uint16)

In [ ]:
plt.imshow(label)

In [ ]:
label.max()

In [ ]:

mask, window, scale, padding = utils.resize_image(
        mask,
        min_dim=256,
        max_dim=256,
        padding=False)

In [ ]:
mask.astype(np.uint8)

In [19]:
from skimage.segmentation import random_walker
l = random_walker(np.ones(shape),label, mode='cg')

C:\ProgramData\Anaconda3\envs\autoscroring\lib\site-packages\ipykernel_launcher.py:2: UserWarning: "cg" mode will be used, but it may be slower than "bf" because SciPy was built without UMFPACK. Consider rebuilding SciPy with UMFPACK; this will greatly accelerate the conjugate gradient ("cg") solver. You may also install pyamg and run the random_walker function in "cg_mg" mode (see docstring).
  
C:\ProgramData\Anaconda3\envs\autoscroring\lib\site-packages\skimage\segmentation\random_walker_segmentation.py:98: RuntimeWarning: divide by zero encountered in double_scalars
  beta /= 10 * data.std()
C:\ProgramData\Anaconda3\envs\autoscroring\lib\site-packages\skimage\segmentation\random_walker_segmentation.py:103: RuntimeWarning: invalid value encountered in multiply
  gradients *= beta


KeyboardInterrupt: 

In [ ]:
plt.imshow(l)

In [ ]:
c[0].numpy().shape

In [ ]:
i = 9
display_instances(dataset.__getitem__(i).numpy().transpose((1,2,0))*255,predictions[i]['boxes'].cpu().numpy(),
                        predictions[i]['masks'].cpu().numpy().squeeze(1).transpose((1,2,0)),
                        predictions[i]['labels'].cpu().numpy(),['bk','nucleus'], predictions[i]['scores'].cpu().numpy())

In [ ]:
import colorsys
import random
from matplotlib import patches,  lines
from skimage.measure import find_contours
from matplotlib.patches import Polygon
def random_colors(N, bright=True):
    """
    Generate random colors.
    To get visually distinct colors, generate them in HSV space then
    convert to RGB.
    """
    brightness = 1.0 if bright else 0.7
    hsv = [(i / N, 1, brightness) for i in range(N)]
    colors = list(map(lambda c: colorsys.hsv_to_rgb(*c), hsv))
    random.shuffle(colors)
    return colors
def apply_mask(image, mask, color, alpha=0.):
    """Apply the given mask to the image.
    """
    for c in range(3):
        image[:, :, c] = np.where(mask == 1,
                                  image[:, :, c] *
                                  (1 - alpha) + alpha * color[c] * 255,
                                  image[:, :, c])
    return image

In [ ]:
def display_instances(image, boxes, masks, class_ids, class_names,
                      scores=None, title="",
                      figsize=(16, 16), ax=None,
                      show_mask=False, show_bbox=True,
                      colors=None, captions=None):
    """
    boxes: [num_instance, (y1, x1, y2, x2, class_id)] in image coordinates.
    masks: [height, width, num_instances]
    class_ids: [num_instances]
    class_names: list of class names of the dataset
    scores: (optional) confidence scores for each box
    title: (optional) Figure title
    show_mask, show_bbox: To show masks and bounding boxes or not
    figsize: (optional) the size of the image
    colors: (optional) An array or colors to use with each object
    captions: (optional) A list of strings to use as captions for each object
    """
    # Number of instances
    N = boxes.shape[0]
    if not N:
        print("\n*** No instances to display *** \n")
    else:
        assert boxes.shape[0] == masks.shape[-1] == class_ids.shape[0]

    # If no axis is passed, create one and automatically call show()
    auto_show = False
    if not ax:
        fig, ax = plt.subplots(1, figsize=figsize)
        auto_show = True

    # Generate random colors
    colors = colors or random_colors(N)

    # Show area outside image boundaries.
    height, width = image.shape[:2]
    ax.set_ylim(height + 10, -10)
    ax.set_xlim(-10, width + 10)
    ax.axis('off')
    ax.set_title(title)

    masked_image = image.astype(np.uint32).copy()
    for i in range(N):
        color = colors[i]

        # Bounding box
        if not np.any(boxes[i]) or scores[i]<=0.6:
            # Skip this instance. Has no bbox. Likely lost in image cropping.
            continue
        x1, y1, x2, y2 = boxes[i]
        if show_bbox:
            p = patches.Rectangle((x1, y1), x2 - x1, y2 - y1, linewidth=2,
                                alpha=0.7, linestyle="dashed",
                                edgecolor=color, facecolor='none')
            ax.add_patch(p)

        # Label
        if not captions:
            class_id = class_ids[i]
            score = scores[i] if scores is not None else None
            label = class_names[class_id]
            caption = "{} {:.3f}".format(label, score) if score else label
        else:
            caption = captions[i]
        ax.text(x1, y1 + 8, caption,
                color='w', size=11, backgroundcolor="none")

        # Mask
        mask = masks[:, :, i]
        if show_mask:
            masked_image = apply_mask(masked_image, mask, color)

        # Mask Polygon
        # Pad to ensure proper polygons for masks that touch image edges.
        padded_mask = np.zeros(
            (mask.shape[0] + 2, mask.shape[1] + 2), dtype=np.uint8)
        padded_mask[1:-1, 1:-1] = (mask>0.5)*255
        contours = find_contours(padded_mask, 2)
        for verts in contours:
            # Subtract the padding and flip (y, x) to (x, y)
            verts = np.fliplr(verts) - 1
            p = Polygon(verts, facecolor="none", edgecolor=color)
            ax.add_patch(p)
    ax.imshow(masked_image.astype(np.uint8))
    plt.savefig('foo.png')
    if auto_show:
        plt.show()